Group:

- Student 1:BA Alhassane
- Student 2:
- Student 3:

# Deep Learning - Lab Exercise 2

**WARNING:** you must have finished the first exercise before this one as you will re-use parts of the code.

In the first lab exercise, we built a simple linear classifier.
Although it can give reasonable results on the MNIST dataset (~92.5% of accuracy), deeper neural networks can achieve more the 99% accuracy on this dataset.
However, it can quickly become really impractical to explicitly code forward and backward passes.
Hence, it is useful to rely on an auto-diff library where we specify the forward pass only, and the backward pass is automatically deduced from the computational graph structure.

In this lab exercise, we will build a small and simple auto-diff lib that mimics the autograd mechanism from Pytorch (of course, we will simplify a lot!)
There is two goal:
- write the missing code
- understand the general structure of an auto-diff lib by looking closely at the code. It will help you to understand how Pytorch works!

In [53]:
# import libs that we will use
import os
import numpy as np
import matplotlib.pyplot as plt
import math

# To load the data we will use the script of Gaetan Marceau Caron
# You can download it from the course webiste and move it to the same directory that contains this ipynb file
import dataset_loader

%matplotlib inline

# Data

In [ ]:
# Download mnist dataset 
if("mnist.pkl.gz" not in os.listdir(".")):
    !wget http://deeplearning.net/data/mnist/mnist.pkl.gz

# if you have it somewhere else, you can comment the lines above
# and overwrite the path below
mnist_path = "./mnist.pkl.gz"

--2021-04-01 23:37:56--  http://deeplearning.net/data/mnist/mnist.pkl.gz
Résolution de deeplearning.net (deeplearning.net)… 132.204.26.28
Connexion à deeplearning.net (deeplearning.net)|132.204.26.28|:80… 

In [ ]:
# load the 3 splits
train_data, dev_data, test_data = dataset_loader.load_mnist(mnist_path)

In [ ]:
index = 900
label = train_data[1][index]
picture = train_data[0][index]

print("label: %i" % label)
plt.imshow(picture.reshape(28,28), cmap='Greys')

# Computation nodes

Instead of directly manipulating numpy arrays, we will manipulate abstraction that contains:
- a value (i.e. a numpy array)
- a bool indicating if we wish to compute the gradient with respect to the value
- the gradient with respect to the value
- the operation to call during backpropagation

There will be two kinds of nodes:
- Tensor: a generic computation node
- Parameter: a computation node that is used to store parameters of the network. Parameters are always leaf nodes, i.e. they cannot be build from other computation nodes.

Our implementation of the backward pass will be really simple and inefficient in the general case.
We will just apply the derivative function for a given tensor and then call the ones of its antecedents, recursively.
This simple algorithm is good enough for this exercise.
You may want to look at the implementation of the relu (given below) to understand what you have to do.

Note that a real implementation of backprop will store temporary values during forward that can be used during backward to improve computation speed. We do not do that here.

**Report bonus:** give an example of computation graph for which this simple backprop implementation will be inefficient.

In [ ]:
class Tensor:
    def __init__(self, data, require_grad=False):
        # test type of data: should be np array
        if isinstance(data, float):
            data = np.array([data,])
        if type(data) != np.ndarray:
            raise RuntimeError("Input should be a numpy array")

        # store data for this tensor
        self.data = data
        self.require_grad = require_grad
        
        # this values should be set to enable autograd!
        self.gradient = None
        self.d = None
        self.backptr = None
        
    def zero_grad(self):
        """
        Set the gradient of thie tensor to 0
        """
        if self.require_grad:
            self.gradient = np.zeros_like(self.data)
            
    def accumulate_gradient(self, gradient):
        """
        Accumulte gradient for this tensor
        """
        if gradient.shape != self.data.shape:
            raise RuntimeError("Invalid gradient dimension")

        if self.gradient is None:
            self.gradient = np.copy(gradient)
        else:
            self.gradient += gradient
            
    def backward(self, g=None):
        """
        The backward pass!
        If g != None, then g is the gradient for the current node.
        i.e. g will be != None only for the loss output.
        
        You should call the function stored in self.d with correct arguments,
        and then recursively call the backward methods of tensors in the backptr list if:
        1. they require a gradient
        2. they are of type Tensor: check with isinstance(o, Tensor)
        """
        if not self.require_grad:  # stop right now if this node does not require a gradient
            return
        
        if g is not None:
            if isinstance(g, float):
                g = np.array([g])
            if type(g) != np.ndarray:
                raise RuntimeError("Gradient should be a numpy array")
            if g.shape != self.data.shape:
                raise RuntimeError("Gradient of different size than the value!")
                
            self.gradient = g

         # TODO...
            #call function d
        if self.d is not None:
            grad = self.d(self.backptr, g)

        #loop over backpointers
        if self.backptr is not None:
            for b in self.backptr:
                #if is Tensor and require_grad is true
                if (type(b) == Tensor) and (b.require_grad):
                    b.backward(grad)
       
        #raise RuntimeError("TODO!")
    
    
class Parameter(Tensor):
    """
    This class will be used to store parameters of the network only!
    """
    def __init__(self, data, name="unamed"):
        super().__init__(data, require_grad=True)
        self.name = name
        
    def backward(self):
        raise RuntimeError("You cannot backprop from a Parameter node")

# Functions

Functions manipulate tensors and build the required information for autograd.
A function returns a Tensor that should have require_grad = True if at least of the arguments require a gradient.

In [ ]:
def any_require_grad(l):
    """
    Input:
    - l: an iterable (e.g. a list)
    Ouput:
    - True if any tensor in the input requires a gradient
    """
    return any(t.require_grad for t in l)

In [ ]:
# Here is an exemple with the ReLU
def relu(x):
    v = np.maximum(0, x.data)
    
    output = Tensor(v, require_grad=x.require_grad)
    output.d = backward_relu
    output.backptr = [x]
    
    return output

def backward_relu(backptr, g):
    x, = backptr
    
    # the gradient is accumulated in the arguments only if required
    if x.require_grad:
        x.accumulate_gradient(g * (x.data > 0))

In [ ]:
def tanh(x):
    # TODO
    v=1.0*(np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
    output = Tensor(v, require_grad=x.require_grad)
    output.d = backward_tanh
    output.backptr = [x]
    
    return output


def backward_tanh(backptr, g):
    # TODO
    x,=backptr
    if(x.require_grad):
        #tanhyperbolique_prime=4.0/(np.exp(x)+ np.exp(x))
        x.accumulate_gradient(g*(x.data>0))

Next, we implement the affine transform operation.
You can reuse the code from the first lab exercise, with one difference: you have to compute the gradient with respect to x too!

In [ ]:
def affine_transform(W, b, x):
    # TODO
    x_transformed=w@x+b
    return x_transformed

def backward_affine_transform(backptr, g):
    # retrieve forward arguments
    W, b, x = backptr
    
    # the gradient is accumulated in the arguments only if required
    if W.require_grad:
        W.accumulate_gradient+=g ## TODO
    if b.require_grad:
        b.accumulate_gradient+=g # TODO
    if x.require_grad:
        #print(W.data.shape, b.data.shape, x.data.shape, g.shape)
        x.accumulate_gradient+=g # TODO

In [ ]:
# we use an underscore because this function does not manipulate tensors:
# it is exactly the same as in the previous exercise
def _softmax(x):
    # TODO
    x=np.array(x)
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

def nll(x, gold):
    # TODO
    v=-x[gold]+ np.log(np.exp(x).sum())
    output = Tensor(v, require_grad=x.require_grad)
    output.d = backward_nll
    output.backptr = [x]
    return output

def backward_nll(backptr, g):
    # TODO
    x, = backptr
    # the gradient is accumulated in the arguments only if required
    if x.require_grad:
        x.accumulate_gradient(g * (x.data > 0))

# Module

Neural networks or parts of neural networks will be stored in Modules.
They implement method to retrieve all parameters of the network and subnetwork.

In [ ]:
class Module:
    def __init__(self):
        raise NotImplemented("")
        
    def parameters(self):
        ret = []
        for name in dir(self):
            o = self.__getattribute__(name)

            if type(o) is Parameter:
                ret.append(o)
            if isinstance(o, Module) or isinstance(o, ModuleList):
                ret.extend(o.parameters())
        return ret

# if you want to store a list of Parameters or Module,
# you must store them in a ModuleList instead of a python list,
# in order to collect the parameters correctly
class ModuleList(list):
    def parameters(self):
        ret = []
        for m in self:
            if type(m) is Parameter:
                ret.append(m)
            elif isinstance(m, Module) or isinstance(m, ModuleList):
                ret.extend(m.parameters())
        return ret

# Initialization and optimization

In [ ]:
def zero_init(b):
    b[:] = 0.

def glorot_init(W):
    # TODO
    size = math.sqrt(6.0/(W.shape[0] + W.shape[1]))
    weights_init = np.random.uniform(-size, size, size=W.shape)
    # TODO
    return weights_init

# Look at slides for the formula!
def kaiming_init(W):
    # TODO
    size = math.sqrt(6.0/(W.shape[0] + W.shape[1]))
    weights_init = np.random.uniform(-size, size, size=W.shape)
    # TODO
    return weights_init

In [ ]:
# simple gradient descent optimizer
class SGD:
    def __init__(self, params, lr=0.1):
        self.params = params
        self.lr = lr
        
    def step(self):
        for p in self.params:
            p.data[:] = p.data - self.lr * p.gradient
        
    def zero_grad(self):
        for p in self.params:
            p.zero_grad()

# Networks and training loop

We first create a simple linear classifier, similar to the first lab exercise.

In [ ]:
class LinearNetwork(Module):
    def __init__(self, dim_input, dim_output):
        # build the parameters
        self.W = Parameter(np.ndarray((dim_output, dim_input)))
        self.b = Parameter(np.ndarray((dim_output,)))
        
        self.init_parameters()
        
    def init_parameters(self):
        # init parameters of the network (i.e W and b)
        glorot_init(self.W.data)
        zero_init(self.b.data)
        
    def forward(self, x):
        return  affine_transform(self.W, self.b, x)

We will train several neural networks.
Therefore, we encapsulate the training loop in a function.

**warning**: you have to call optimizer.zero_grad() before each backward pass to reinitialize the gradient of the parameters!

In [ ]:
def training_loop(network, optimizer, train_data, dev_data, n_epochs=10):
    # TODO!
    # almost the same as last week
    n=0
    for epoch in range(n_epochs):
        # TODO# ...
        theta = [W,b]
        best_acc = 0
        acc_train = []
        acc_dev = []
        X_train = train_data[0]
        Y_train = train_data[1]
        for i in range(0,getDimDataset(train_data)[0]):
            x_row = X_train[i]
            gold = Y_train[i]
            #forward pass
            x_row_tranformed = affine_transform(W, b, x_row)
            y_gold = np.argmax(  x_row_tranformed)
            if y_gold == gold:
                n+=1
            #training
            optimizer.zero_grad() 
            g_x = backward_nll( x_row_tranformed, gold, 1.)
            g_W, g_b = backward_affine_transform(W, b, x_row, g_x)
            W = W - step*g_W
            b = b - step*g_b
    theta=[W,b]         
    return theta

In [ ]:
dim_input = 28*28
dim_output = 10

network = LinearNetwork(dim_input, dim_output)
#data, require_grad=True
optimizer = SGD(network.parameters(), 0.01)

training_loop(network, optimizer, train_data, dev_data, n_epochs=5)

After you finished the linear network, you can move to a deep network!
The tanh argument can be used to switch between relu or tanh. Warning: use the correct initialization for a given activation!

In [ ]:
class DeepNetwork(Module):
    def __init__(self, dim_input, dim_output, hidden_dim, n_layers, tanh=False):
        self.W = ModuleList()
        self.b = ModuleList()
        
        # TODO...
        
        self.output_proj = # TODO
        self.output_bias = # TODO
        
        self.init_parameters()
        
    def init_parameters(self):
        # TODO...

    def forward(self, x):
        # TODO

In [ ]:
dim_input = 28*28
dim_output = 10

network = DeepNetwork(dim_input, dim_output, 100, 1, tanh=False)
optimizer = SGD(network.parameters(), 0.01)

training_loop(network, optimizer, train_data, dev_data, n_epochs=10)

In [ ]:
dim_input = 28*28
dim_output = 10

network = DeepNetwork(dim_input, dim_output, 100, 1, tanh=True)
optimizer = SGD(network.parameters(), 0.01)

training_loop(network, optimizer, train_data, dev_data, n_epochs=10)

# Bonus

You can try to implement a momentum SGD optimizer!
Note that you have to keep track of the velocity for each parameter in the optimizer.